# **Google Stock Price Prediction Using LSTM**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Dataset/Stock_Price_Train.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
training_set = train.iloc[:, 1:2].values

In [ ]:
training_set

In [ ]:
training_set.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))
scaled_training_set = sc.fit_transform(training_set)

In [ ]:
scaled_training_set

In [ ]:
scaled_training_set.shape

In [ ]:
X_train = []
Y_train = []

for i in range(60, 1258):
    X_train.append(scaled_training_set[i-60:i, 0])
    Y_train.append(scaled_training_set[i, 0])
X_train, Y_train = np.array(X_train), np.array(Y_train)

In [ ]:
X_train.shape

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Dataset/Stock_Price_Test.csv')
real_stock_price = test.iloc[:, 1:2].values
total_data = pd.concat([train['Open'], test['Open']], axis=0)

In [ ]:
inputs = total_data[len(total_data)-len(test)-60:].values
print(inputs.shape)

In [ ]:
inputs = inputs.reshape(-1, 1)
print(inputs.shape)

In [ ]:
inputs = sc.transform(inputs)

In [ ]:
X_test = []

for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])

In [ ]:
X_test = np.array(X_test)
print(X_test.shape)

In [ ]:
X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)

In [ ]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.figure(figsize=[8, 6])
plt.plot(predicted_stock_price, 'r', label='Predicted Stock Price', linewidth=3.0)
plt.plot(real_stock_price, 'b',label='Real Stock Price', linewidth=3.0)
plt.legend()
plt.xlabel('Time', fontsize=16)
plt.ylabel('Stock Price', fontsize=16)
plt.title('Google Stock Price Prediction', fontsize=16)
plt.show()